In [15]:
from datasets import load_from_disk, load_dataset

In [17]:
#dataset = load_dataset('../datasets/base/dataset_fixed_v3', data_files={"train": "train.csv", "validation": "validation.csv", "test": "test.csv"})

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

/home/boa/.conda/envs/ai/lib/python3.10/site-packages/datasets/download/streaming_download_manager.py:776: FutureWarning: the 'mangle_dupe_cols' keyword is deprecated and will be removed in a future version. Please take steps to stop the use of 'mangle_dupe_cols'
  return pd.read_csv(xopen(filepath_or_buffer, "rb", use_auth_token=use_auth_token), **kwargs)


Generating validation split: 0 examples [00:00, ? examples/s]

/home/boa/.conda/envs/ai/lib/python3.10/site-packages/datasets/download/streaming_download_manager.py:776: FutureWarning: the 'mangle_dupe_cols' keyword is deprecated and will be removed in a future version. Please take steps to stop the use of 'mangle_dupe_cols'
  return pd.read_csv(xopen(filepath_or_buffer, "rb", use_auth_token=use_auth_token), **kwargs)


Generating test split: 0 examples [00:00, ? examples/s]

/home/boa/.conda/envs/ai/lib/python3.10/site-packages/datasets/download/streaming_download_manager.py:776: FutureWarning: the 'mangle_dupe_cols' keyword is deprecated and will be removed in a future version. Please take steps to stop the use of 'mangle_dupe_cols'
  return pd.read_csv(xopen(filepath_or_buffer, "rb", use_auth_token=use_auth_token), **kwargs)


Dataset csv downloaded and prepared to /home/boa/.cache/huggingface/datasets/csv/dataset_fixed_v3-60856dafd072468a/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [28]:
dataset = load_dataset('boapps/kmdb_base', revision="caa20c8f62b6e5320673c5194c8a75e93515f341")

Found cached dataset parquet (/home/boa/.cache/huggingface/datasets/boapps___parquet/boapps--kmdb_base-daedf5fec5a8c1a2/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)


  0%|          | 0/3 [00:00<?, ?it/s]

In [29]:
dataset

DatasetDict({
    validation: Dataset({
        features: ['text', 'title', 'description', 'keywords', 'url', 'authors', 'date', 'kmonitor_description', 'kmonitor_title', 'kmonitor_tags', 'kmonitor_institutes', 'kmonitor_people', 'kmonitor_places', 'kmonitor_collections'],
        num_rows: 3410
    })
    test: Dataset({
        features: ['text', 'title', 'description', 'keywords', 'url', 'authors', 'date', 'kmonitor_description', 'kmonitor_title', 'kmonitor_tags', 'kmonitor_institutes', 'kmonitor_people', 'kmonitor_places', 'kmonitor_collections'],
        num_rows: 1720
    })
    train: Dataset({
        features: ['text', 'title', 'description', 'keywords', 'url', 'authors', 'date', 'kmonitor_description', 'kmonitor_title', 'kmonitor_tags', 'kmonitor_institutes', 'kmonitor_people', 'kmonitor_places', 'kmonitor_collections'],
        num_rows: 26661
    })
})

In [30]:
import re

html_pattern = re.compile(r'<.*?>')
date_pattern = re.compile(r'\d{4}\-\d{2}\-\d{2} \d{2}:\d{2}')
slug_pattern = re.compile(r' \[sharedcontent .*\]')

def clean_article(article):
    article['description'] = html_pattern.sub('', article['description'])
    article['description'] = date_pattern.sub('', article['description'])
    article['description'] = slug_pattern.sub('', article['description'])
    article['description'] = article['description'].replace(r'\xa0', ' ')
    article['description'] = article['description'].replace(r'Ã¶', 'ö')
    article['description'] = article['description'].replace(r'Ã©', 'é')
    article['description'] = article['description'].replace(r'Ã³', 'ó')
    article['description'] = article['description'].replace(r'Ã¡', 'á')
    article['description'] = article['description'].replace(r'Ã\xad', 'í')
    article['description'] = article['description'].replace(r'Ã¼', 'ü')
    article['description'] = article['description'].replace(r'Ã¼', 'ü')
    article['description'] = article['description'].replace(r'+ Videó', '')
    article['description'] = article['description'].replace(r'•', '')
    article['description'] = article['description'].replace(r'MNO - PL', ' ')
    article['description'] = article['description'].replace('\\\\', '')
    article['description'] = article['description'].strip('- ')

    article['title'] = article['title'].replace(r'Belföld :: ', '')
    article['title'] = article['title'].replace(r' (+videó)', '')
    article['title'] = article['title'].replace('\\\\', '')

    return article

dataset = dataset.map(clean_article)
dataset = dataset.filter(lambda row: row['description'] != '')
RE_SUSPICIOUS = re.compile(r'[#<>{}\|\=\*\[\]\\]')
field = 'description'
dataset = dataset.map(lambda row: {'score': len(RE_SUSPICIOUS.findall(row[field]))/len(row[field])})
dataset = dataset.map(lambda row: {'title_score': len(RE_SUSPICIOUS.findall(row['title']))/len(row['title'])})

dataset.filter(lambda row: row['title_score'] > 0.01)['train']['title']

dataset.filter(lambda row: row['title_score'] > 0.01)['train']['title']
dataset = dataset.filter(lambda row: row['score'] < 0.01)
dataset = dataset.filter(lambda row: row['title_score'] < 0.01)

Loading cached processed dataset at /home/boa/.cache/huggingface/datasets/boapps___parquet/boapps--kmdb_base-daedf5fec5a8c1a2/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec/cache-0b4c562a1c4d00ab.arrow
Loading cached processed dataset at /home/boa/.cache/huggingface/datasets/boapps___parquet/boapps--kmdb_base-daedf5fec5a8c1a2/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec/cache-667f4b06a78b6966.arrow
Loading cached processed dataset at /home/boa/.cache/huggingface/datasets/boapps___parquet/boapps--kmdb_base-daedf5fec5a8c1a2/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec/cache-b8aa6fc4806e321f.arrow
Loading cached processed dataset at /home/boa/.cache/huggingface/datasets/boapps___parquet/boapps--kmdb_base-daedf5fec5a8c1a2/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec/cache-42e14dfbe3efe1c3.arrow
Loading cached processed dataset at /home/boa/.cache/huggingface/datasets/boapps___parquet/boapp

In [31]:
dataset

DatasetDict({
    validation: Dataset({
        features: ['text', 'title', 'description', 'keywords', 'url', 'authors', 'date', 'kmonitor_description', 'kmonitor_title', 'kmonitor_tags', 'kmonitor_institutes', 'kmonitor_people', 'kmonitor_places', 'kmonitor_collections', 'score', 'title_score'],
        num_rows: 3397
    })
    test: Dataset({
        features: ['text', 'title', 'description', 'keywords', 'url', 'authors', 'date', 'kmonitor_description', 'kmonitor_title', 'kmonitor_tags', 'kmonitor_institutes', 'kmonitor_people', 'kmonitor_places', 'kmonitor_collections', 'score', 'title_score'],
        num_rows: 1710
    })
    train: Dataset({
        features: ['text', 'title', 'description', 'keywords', 'url', 'authors', 'date', 'kmonitor_description', 'kmonitor_title', 'kmonitor_tags', 'kmonitor_institutes', 'kmonitor_people', 'kmonitor_places', 'kmonitor_collections', 'score', 'title_score'],
        num_rows: 26534
    })
})

In [33]:
base_dataset = dataset.remove_columns(['title_score', 'score'])

In [ ]:
classification_dataset = load_from_disk('../datasets/base/kmdb_classification_extended')

In [ ]:
classification_dataset

In [ ]:
classification_dataset = classification_dataset.remove_columns(['title_score', 'score'])

In [ ]:
classification_dataset

In [34]:
base_dataset.push_to_hub("boapps/kmdb_base")

Pushing split validation to the Hub.


Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/4 [00:00<?, ?ba/s]

Deleting unused files from dataset repository:   0%|          | 0/1 [00:00<?, ?it/s]

Pushing split test to the Hub.


...

Deleting unused files from dataset repository:   0%|          | 0/1 [00:00<?, ?it/s]

Pushing split train to the Hub.


Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/27 [00:00<?, ?ba/s]

Deleting unused files from dataset repository:   0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
#classification_dataset.push_to_hub("boapps/kmdb_classification")